<a href="https://colab.research.google.com/github/skj092/Object-Detection/blob/main/notebooks/TFOD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Copyright 2020 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/tf2_object_detection"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf2_object_detection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/tf2_object_detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/tf2_object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
    <a href="https://tfhub.dev/tensorflow/collections/object_detection/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />See TF Hub models</a>
  </td>
</table>

# TensorFlow Hub Object Detection Colab

Welcome to the TensorFlow Hub Object Detection Colab! This notebook will take you through the steps of running an "out-of-the-box" object detection model on images.

### More models
[This](https://tfhub.dev/tensorflow/collections/object_detection/1) collection contains TF2 object detection models that have been trained on the COCO 2017 dataset. [Here](https://tfhub.dev/s?module-type=image-object-detection) you can find all object detection models that are currently hosted on [tfhub.dev](https://tfhub.dev/).

## Imports and Setup

Let's start with the base imports.

In [ ]:
!pip install -q kaggle 
!pip install -q kaggle-cli

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = 'sonujha090'
os.environ['KAGGLE_KEY'] = "************************"

In [ ]:
!kaggle datasets download -d mbkinaci/fruit-images-for-object-detection 

In [ ]:
!unzip /content/fruit-images-for-object-detection.zip

In [ ]:
# # This Colab requires TF 2.5.
# !pip install -U "tensorflow>=2.5"

In [ ]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import cv2
import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

In [ ]:
%%bash

# Clone the tensorflow models repository
git clone --depth 1 https://github.com/tensorflow/models

**Setting up the directory structure**

In [ ]:
%%bash

mkdir TensorFlow

mkdir TensorFlow/workspace

mkdir TensorFlow/script

mkdir TensorFlow/script/preprocessing

mkdir TensorFlow/workspace/training_demo

mkdir TensorFlow/workspace/training_demo/annotations/

mkdir TensorFlow/workspace/training_demo/exported-models

mkdir TensorFlow/workspace/training_demo/images

mkdir TensorFlow/workspace/training_demo/models

mkdir TensorFlow/workspace/training_demo/pre-trained-models

mv models TensorFlow/

mv train_zip/train /content/TensorFlow/workspace/training_demo/images

mv test_zip/test /content/TensorFlow/workspace/training_demo/images

In [ ]:
!cp /content/TensorFlow/models/research/object_detection/data/mscoco_label_map.pbtxt /content/TensorFlow/workspace/training_demo/annotations/

In [ ]:
# uodate mscoco_label_map.pbtxt


label_map_str = """item {
    id: 1
    name: 'apple'
}

item {
    id: 2
    name: 'banana'
}

item {
    id: 3
    name: 'orange'
}"""

with open('/content/TensorFlow/workspace/training_demo/annotations/mscoco_label_map.pbtxt', 'w') as f:
  f.write(label_map_str)

!more /content/TensorFlow/workspace/training_demo/annotations/mscoco_label_map.pbtxt

In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd /content/TensorFlow/models/research
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

In [ ]:
%%bash

wget "https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py" 



In [ ]:
mv generate_tfrecord.py /content/TensorFlow/script/preprocessing

In [ ]:
cd /content/TensorFlow/script/preprocessing

In [ ]:
!python generate_tfrecord.py -x /content/TensorFlow/workspace/training_demo/images/train -l /content/TensorFlow/workspace/training_demo/annotations/mscoco_label_map.pbtxt -o /content/TensorFlow/workspace/training_demo/annotations/train.record


In [ ]:
!python generate_tfrecord.py -x /content/TensorFlow/workspace/training_demo/images/test -l /content/TensorFlow/workspace/training_demo/annotations/mscoco_label_map.pbtxt -o /content/TensorFlow/workspace/training_demo/annotations/test.record

In [ ]:
!wget "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz"

In [ ]:
!tar -xvf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8 /content/TensorFlow/workspace/training_demo/pre-trained-models

In [ ]:
!mkdir /content/TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn/

In [ ]:
!cp /content/TensorFlow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config /content/TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn/

In [ ]:
from string import Template

config_file_template =  """model {
  ssd {
    num_classes: 3
    image_resizer {
      fixed_shape_resizer {
        height: 640
        width: 640
      }
    }
    feature_extractor {
      type: "ssd_resnet50_v1_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 0.00039999998989515007
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999329447746
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 0.00039999998989515007
            }
          }
          initializer {
            random_normal_initializer {
              mean: 0.0
              stddev: 0.009999999776482582
            }
          }
          activation: RELU_6
          batch_norm {
            decay: 0.996999979019165
            scale: true
            epsilon: 0.0010000000474974513
          }
        }
        depth: 256
        num_layers_before_predictor: 4
        kernel_size: 3
        class_prediction_bias_init: -4.599999904632568
      }
    }
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 4.0
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        scales_per_octave: 2
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 9.99999993922529e-09
        iou_threshold: 0.6000000238418579
        max_detections_per_class: 100
        max_total_detections: 100
        use_static_shapes: false
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 2.0
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    encode_background_as_zeros: true
    normalize_loc_loss_by_codesize: true
    inplace_batchnorm_update: true
    freeze_batchnorm: false
  }
}
train_config {
  batch_size: 8
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_crop_image {
      min_object_covered: 0.0
      min_aspect_ratio: 0.75
      max_aspect_ratio: 3.0
      min_area: 0.75
      max_area: 1.0
      overlap_thresh: 0.0
    }
  }
  sync_replicas: true
  optimizer {
    momentum_optimizer {
      learning_rate {
        cosine_decay_learning_rate {
          learning_rate_base: 0.03999999910593033
          total_steps: $training_steps
          warmup_learning_rate: 0.013333000242710114
          warmup_steps: $warmup_steps
        }
      }
      momentum_optimizer_value: 0.8999999761581421
    }
    use_moving_average: false
  }
  fine_tune_checkpoint: "/content/TensorFlow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0"
  num_steps: $training_steps #set the steps
  startup_delay_steps: 0.0
  replicas_to_aggregate: 8
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection"
  use_bfloat16: false
  fine_tune_checkpoint_version: V2
}
train_input_reader {
  label_map_path: "/content/TensorFlow/workspace/training_demo/annotations/mscoco_label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/content/TensorFlow/workspace/training_demo/annotations/train.record"
  }
}
eval_config {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
}
eval_input_reader {
  label_map_path: "/content/TensorFlow/workspace/training_demo/annotations/mscoco_label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/TensorFlow/workspace/training_demo/annotations/test.record"
  }
}

"""

In [ ]:
# Define the training pipeline

TRAINING_STEPS = 2000
WARMUP_STEPS = 200
PIPELINE_CONFIG_PATH='dataset/pipeline.config'

pipeline = Template(config_file_template).substitute(
    training_steps=TRAINING_STEPS, warmup_steps=WARMUP_STEPS)

with open("/content/TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn/pipeline.config", 'w') as f:
    f.write(pipeline)

In [ ]:
# # pipeline.config changes

# 3 : 3
# 131 8
# 161: "/content/TensorFlow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0"
# 167: 'detection'
# 168: false
# 172: "/content/TensorFlow/workspace/training_demo/annotations/mscoco_label_map.pbtxt"
# 174: "/content/TensorFlow/workspace/training_demo/annotations/train.record"
# 182: "/content/TensorFlow/workspace/training_demo/annotations/mscoco_label_map.pbtxt"
# 186: "/content/TensorFlow/workspace/training_demo/annotations/test.record"

In [ ]:
!cp /content/TensorFlow/models/research/object_detection/model_main_tf2.py /content/TensorFlow/workspace/training_demo

In [ ]:
cd /content/TensorFlow/workspace/training_demo

In [ ]:
!pip uninstall opencv-python-headless -y

In [ ]:
!pip install opencv-python-headless==4.1.2.30

In [ ]:
!cp /content/TensorFlow/models/research/object_detection/model_main_tf2.py /content/TensorFlow/workspace/training_demo

In [ ]:
cd /content/TensorFlow/workspace/training_demo

In [ ]:
train_record_path = '/content/TensorFlow/workspace/training_demo/annotations/train.record'
test_record_path = '/content/TensorFlow/workspace/training_demo/annotations/train.record'
model_dir = '/content/TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn'
labelmap_path = '/content/TensorFlow/workspace/training_demo/annotations/mscoco_label_map.pbtxt'

pipeline_config_path = '/content/TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn/pipeline.config'
fine_tune_checkpoint = '/content/TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn/ckpt-1'

In [ ]:
!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config

**Validation on test dataset**

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --checkpoint_dir={model_dir} 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn/'

In [ ]:
! cp /content/TensorFlow/models/research/object_detection/exporter_main_v2.py /content/TensorFlow/workspace/training_demo

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path /content/TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir /content/TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn --output_directory trained-inference-graphs/output


# Inference

In [ ]:
from glob import glob
test_images = glob('/content/TensorFlow/workspace/training_demo/images/test/*.jpg')
test_images[:5]

In [ ]:
import time
import os 
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

PATH_TO_MODEL_DIR = "/content/TensorFlow/workspace/training_demo/trained-inference-graphs/output"

PATH_TO_CFG = PATH_TO_MODEL_DIR + "/pipeline.config"
PATH_TO_CKPT = PATH_TO_MODEL_DIR + "/checkpoint"

print('Loading model... ', end='')
start_time = time.time()

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

In [ ]:
PATH_TO_LABELS = "/content/TensorFlow/workspace/training_demo/annotations/mscoco_label_map.pbtxt"
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_path in test_images[:3]:

    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=.30,
            agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
plt.show()

In [ ]:
def nms(rects, thd=0.5):
    """
    Filter rectangles
    rects is array of oblects ([x1,y1,x2,y2], confidence, class)
    thd - intersection threshold (intersection divides min square of rectange)
    """
    out = []

    remove = [False] * len(rects)

    for i in range(0, len(rects) - 1):
        if remove[i]:
            continue
        inter = [0.0] * len(rects)
        for j in range(i, len(rects)):
            if remove[j]:
                continue
            inter[j] = intersection(rects[i][0], rects[j][0]) / min(square(rects[i][0]), square(rects[j][0]))

        max_prob = 0.0
        max_idx = 0
        for k in range(i, len(rects)):
            if inter[k] >= thd:
                if rects[k][1] > max_prob:
                    max_prob = rects[k][1]
                    max_idx = k

        for k in range(i, len(rects)):
            if (inter[k] >= thd) & (k != max_idx):
                remove[k] = True

    for k in range(0, len(rects)):
        if not remove[k]:
            out.append(rects[k])

    boxes = [box[0] for box in out]
    scores = [score[1] for score in out]
    classes = [cls[2] for cls in out]
    return boxes, scores, classes


def intersection(rect1, rect2):
    """
    Calculates square of intersection of two rectangles
    rect: list with coords of top-right and left-boom corners [x1,y1,x2,y2]
    return: square of intersection
    """
    x_overlap = max(0, min(rect1[2], rect2[2]) - max(rect1[0], rect2[0]));
    y_overlap = max(0, min(rect1[3], rect2[3]) - max(rect1[1], rect2[1]));
    overlapArea = x_overlap * y_overlap;
    return overlapArea


def square(rect):
    """
    Calculates square of rectangle
    """
    return abs(rect[2] - rect[0]) * abs(rect[3] - rect[1])

In [ ]:
def inference_as_raw_output(path2images,
                            box_th = 0.25,
                            nms_th = 0.5,
                            to_file = False,
                            data = None,
                            path2dir = False):
  print (f'Current data set is {data}')
  """
  Function that performs inference and return filtered predictions
  
  Args:
    path2images: an array with pathes to images
    box_th: (float) value that defines threshold for model prediction. Consider 0.25 as a value.
    nms_th: (float) value that defines threshold for non-maximum suppression. Consider 0.5 as a value.
    to_file: (boolean). When passed as True => results are saved into a file. Writing format is
    path2image + (x1abs, y1abs, x2abs, y2abs, score, conf) for box in boxes
    data: (str) name of the dataset you passed in (e.g. test/validation)
    path2dir: (str). Should be passed if path2images has only basenames. If full pathes provided => set False.
    
  Returs:
    detections (dict): filtered predictions that model made
  """
  print (f'Current data set is {data}')
  print (f'Ready to start inference on {len(path2images)} images!')
  
  for image_path in tqdm(path2images):
      
      if path2dir: # if a path to a directory where images are stored was passed in
          image_path = os.path.join(path2dir, image_path.strip())
          
      image_np = load_image_into_numpy_array(image_path)

      input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
      detections = detect_fn(input_tensor)
      
      # checking how many detections we got
      num_detections = int(detections.pop('num_detections'))
      
      # filtering out detection in order to get only the one that are indeed detections
      detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
      
      # detection_classes should be ints.
      detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
      
      # defining what we need from the resulting detection dict that we got from model output
      key_of_interest = ['detection_classes', 'detection_boxes', 'detection_scores']
      
      # filtering out detection dict in order to get only boxes, classes and scores
      detections = {key: value for key, value in detections.items() if key in key_of_interest}
      
      if box_th: # filtering detection if a confidence threshold for boxes was given as a parameter
          for key in key_of_interest:
              scores = detections['detection_scores']
              current_array = detections[key]
              filtered_current_array = current_array[scores > box_th]
              detections[key] = filtered_current_array
      
      if nms_th: # filtering rectangles if nms threshold was passed in as a parameter
          # creating a zip object that will contain model output info as
          output_info = list(zip(detections['detection_boxes'],
                                  detections['detection_scores'],
                                  detections['detection_classes']
                                )
                            )
          boxes, scores, classes = nms(output_info)
          
          detections['detection_boxes'] = boxes # format: [y1, x1, y2, x2]
          detections['detection_scores'] = scores
          detections['detection_classes'] = classes
          
      if to_file and data: # if saving to txt file was requested

          image_h, image_w, _ = image_np.shape
          file_name = f'pred_result_{data}.txt'
          
          line2write = list()
          line2write.append(os.path.basename(image_path))
          
          with open(file_name, 'a+') as text_file:
              # iterating over boxes
              for b, s, c in zip(boxes, scores, classes):
                  
                  y1abs, x1abs = b[0] * image_h, b[1] * image_w
                  y2abs, x2abs = b[2] * image_h, b[3] * image_w
                  
                  list2append = [x1abs, y1abs, x2abs, y2abs, s, c]
                  line2append = ','.join([str(item) for item in list2append])
                  
                  line2write.append(line2append)
              
              line2write = ' '.join(line2write)
              text_file.write(line2write + os.linesep)
      
      return detections

In [ ]:
# https://app.neptune.ai/anton-morgunov/tf-test/n/model-for-inference-36c9b0c4-8d20-4d5a-aa54-5240cc8ce764/6f67c0e3-283c-45de-ae56-405aecd736c0